In [1]:
import pandas as pd

df = pd.read_csv('/Users/simonezacchetti/Documents/università/magistrale/secondo anno/nlp/cooking-mama/full_dataset.csv')

print(df.iloc[2081257])

Unnamed: 0                                               2081257
title                                            Kale Apple Love
ingredients    ["3 Kale leaves", "5 small apples or", "1/2 Le...
directions                                             ["Juice"]
link               cookpad.com/us/recipes/356880-kale-apple-love
source                                                 Recipes1M
NER                      ["leaves", "apples", "Lemon", "apples"]
Name: 2081257, dtype: object
